In [16]:
import csv
import pandas as pd
from datetime import date, datetime

In [8]:
failed = pd.read_csv("data/fdic/failures.csv", encoding='latin1')
all = pd.read_csv("data/fdic/institutions.csv", encoding='latin1')

/var/folders/t1/wlm1x_7n48l030ngp1yv8sdw0000gn/T/ipykernel_39692/3003168610.py:2: DtypeWarning: Columns (92,93,94,95,96,97) have mixed types. Specify dtype option on import or set low_memory=False.
  all = pd.read_csv("data/fdic/institutions.csv", encoding='latin1')


Index(['STNAME', 'CERT', 'DOCKET', 'ACTIVE', 'ADDRESS', 'ASSET', 'BKCLASS',
       'CHANGEC1', 'CHANGEC2', 'CHANGEC3',
       ...
       'CBSA', 'CBSA_NO', 'CBSA_METRO_NAME', 'CBSA_METRO', 'CBSA_METRO_FLG',
       'CBSA_MICRO_FLG', 'CBSA_DIV', 'CBSA_DIV_NO', 'CBSA_DIV_FLG', 'CB'],
      dtype='object', length=142)

I then filter the institutions data from 142 columns (variables) down to columns I want to focus on. Selection is based on literature review and previous research 
See institutions_definitions.csv for more detailed variable defintiions for all 142 columns. Here are the noteable ones I chose to include:

* CERT: FDIC Certificate, a unique number assigned by the FDIC used to identify institutions and for the issuance of insurance certificates.
* ACTIVE: Institution Status, a number indicating the status of an institution. 
                                1='Institutions that are currently open and insured by the FDIC'
                                0='Institution closed or not insured by FDIC'
* BKCLASS: Institution Class, a classification code assigned by the FDIC based on the institution's charter type (commercial bank or savings institution), 
                                charter agent (state or federal), Federal Reserve membership status (Fed member, Fed non-member) and its primary federal 
                                regulator (state chartered institutions are subject to both federal and state supervision). 
    N - Commercial bank, national (federal) charter, Fed member, and supervised by the Office of the Comptroller of the Currency (OCC)
    NM - Commercial bank, state charter, Fed non-member, and supervised by the Federal Deposit Insurance Corporation (FDIC)
    OI - Insured U.S. branch of a foreign chartered institution (IBA) and supervised by the OCC or FDIC
    SB - Federal savings banks, federal charter, supervised by the OCC or before July 21,2011 the Office of Thrift Supervision (OTS)
    SI - State chartered stock savings banks, supervised by the FDIC
    SL - State chartered stock savings and loan associations, supervised by the FDIC or before July 21,2011 the OTS
    SM - Commercial bank, state charter, Fed member, and supervised by the Federal Reserve Bank (FRB)
    NC - Noninsured non-deposit commercial banks and/or trust companies regulated by the OCC, a state, or a territory
    NS - Noninsured stock savings bank supervised by a state or territory
    CU - state or federally chartered credit unions supervised by the National Credit Union Association (NCUA).
* ESTYMD: Established Date, the date on which the institution or branch began operations.
* ASSET: Total assets, the sum of all assets owned by the institution including cash, loans, securities, bank premises and other assets. This total does not include off-balance-sheet accounts.
* EQ: Equity capital, total equity capital (includes preferred and common stock, surplus and undivided profits).
* DEP: total deposits, the sum of all deposits including demand deposits, money market deposits, other savings deposits, time deposits and deposits in foreign offices.
* OFFICES: A branch/office is any location, or facility, of a financial institution, including its main office, where deposit accounts are opened, deposits are accepted,
             checks paid, and loans granted. Some branches include, but are not limited to, brick and mortar locations, detached or attached drive-in facilities, seasonal 
             offices, offices on military bases or government installations, paying/receiving stations or units, nondeposit offices, Internet and PhoneBanking locations 
             where a customer can open accounts, make deposits and borrow money. A branch does not include Automated Teller Machines (ATM), Consumer Credit Offices, 
             Contractual Offices, Customer Bank Communication Terminals (CBCT), Electronic Fund Transfer Units (EFTU), and Loan Production Offices Summary of Deposits 
             information is required for each insured office located in any State, the District of Columbia, the Commonwealth of Puerto Rico or any U.S. territory or 
             possession such as Guam or the U.S. Virgin Islands, without regard to the location of the main office.
* ROA: Return on assets (ROA), net income after taxes and extraordinary items (annualized) as a percent of average total assets.
* NETINC: Net income, net interest income plus total noninterest income plus realized gains (losses) on securities and extraordinary items, less total noninterest expense, 
            loan loss provisions and income taxes.
* MUTUAL: Ownership Type, banking institutions fall into one of two ownership types, stock or non-stock. An institution which sells stock to raise capital is called a stock
            institution. It is owned by the shareholders who benefit from profits earned by the institution. A non-stock institution, or mutual institution, is owned
            and controlled solely by its depositors. A mutual does not issue capital stock.


In [10]:
cols = ["CERT", "NAME", "STNAME", "CITY", "ACTIVE", "BKCLASS", "ESTYMD","ASSET","NETINC", "EQ", "DEP", "ROA","MUTUAL","OFFICES"]
data = all[cols]
data


,CERT,NAME,STNAME,CITY,ACTIVE,BKCLASS,ESTYMD,ASSET,NETINC,EQ,DEP,ROA,MUTUAL,OFFICES
0,20671,Security Bank and Trust Company,Rhode Island,North Providence,0,NM,06/05/1972,44943.0,301.0,3939.0,40106.0,1.395349,0.0,NaN
1,20679,NorthWest Bank,Washington,Seattle,0,NM,05/01/1972,81258.0,504.0,5817.0,74980.0,1.248818,0.0,NaN
2,23833,"Bank of Tyler, National Association",Texas,Tyler,0,N,04/16/1982,8558.0,-95.0,833.0,7411.0,-0.955100,0.0,NaN
3,23839,The Ken-Caryl Bank,Colorado,Littleton,0,NM,05/17/1982,26114.0,39.0,1452.0,18939.0,0.162813,0.0,NaN
4,23840,"Mercantile Bank of South County, National Asso...",Missouri,St. Louis County,0,N,05/21/1982,27198.0,-108.0,2230.0,24745.0,-0.446314,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27820,35090,First Community Bank,Louisiana,Hammond,0,NM,01/19/2000,105580.0,-89.0,12315.0,84384.0,-0.336068,0.0,NaN
27821,7748,VIST Bank,Pennsylvania,Wyomissing,0,NM,04/08/1891,1945388.0,11546.0,199110.0,1717486.0,0.579254,0.0,NaN
27822,5439,Minden Exchange Bank & Trust Company,Nebraska,Minden,1,NM,09/01/1883,195893.0,2625.0,29280.0,135286.0,1.819855,0.0,1.0
27823,90240,Merrimac Savings Bank,Massachusetts,Merrimac,0,SI,04/19/1871,75190.0,203.0,5091.0,70028.0,0.362851,1.0,NaN


Data cleanup: 

In [35]:
current_date = datetime(2024, 1, 1)
date_format = "%m/%d/%Y"

# convert string to date object
data['ESTYMD'] = pd.to_datetime(data['ESTYMD'], format="%m/%d/%Y")

# new column
data['AGE'] = (current_date - data['ESTYMD']).dt.days
data['AGE']

/var/folders/t1/wlm1x_7n48l030ngp1yv8sdw0000gn/T/ipykernel_39692/1990215434.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ESTYMD'] = pd.to_datetime(data['ESTYMD'], format="%m/%d/%Y")
/var/folders/t1/wlm1x_7n48l030ngp1yv8sdw0000gn/T/ipykernel_39692/1990215434.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['AGE'] = (current_date - data['ESTYMD']).dt.days


0        18837
1        18872
2        15235
3        15204
4        15200
         ...  
27820     8748
27821    48480
27822    51256
27823    55774
27824    27876
Name: AGE, Length: 27825, dtype: int64